In [3]:
import datetime

event_signifiers = {
    'Charging': ['Charging', 'discharging'],
    'Home': ['Arrive at Home', 'leave Home'],
    'Heidi': ["Arrived at Heidi's", "Left Heidi's"],
    'wifi': ['Connected to wifi', 'Disconnected from wifi'],
}

def parse_events(raw_logs):
  print(f"Parsing {len(raw_logs)} raw events")
  events = []

  for i in range(len(raw_logs)):
    
    start_line = raw_logs[i]
    if len(start_line) < 3:
        continue
    
    for event_name, signifiers in event_signifiers.items():
        if signifiers[0] in start_line[2]:
            # try to find the next discharging event
            for j in range(i + 1, len(raw_logs)):
                end_line = raw_logs[j]
                if len(end_line) < 3:
                    continue
                if signifiers[1] in end_line[2]:
                    # found a charging event
                    datetime_format = '%Y-%m-%d %I:%M:%S %p %Z'
                    start = datetime.datetime.strptime(start_line[0] + ' ' + start_line[1], datetime_format)
                    end = datetime.datetime.strptime(end_line[0] + ' ' + end_line[1], datetime_format)

                    events.append({
                        'name': event_name,
                        'start': start,
                        'end': end,
                        'eventType': event_name.lower()
                    })
                    break
                if end_line[2] == signifiers[0]:
                    break
  return events
        
root_dir = "/Users/nshelton/Library/Mobile Documents/iCloud~is~workflow~my~workflows/Documents"
import os


all_file_contents = []

start_date = datetime.date(2024, 1, 1)
current_date = start_date

for i in range(365):
    
    day_string = current_date.isoformat()
    filePath = f"{root_dir}/{day_string}.txt"
    if os.path.exists(filePath):
        with open(filePath, 'r') as f:
            fileContent = f.read()
            lines = fileContent.split('\n')
            rawFileContent = [[day_string] + line.split(',') for line in lines]
            all_file_contents += rawFileContent


    current_date += datetime.timedelta(days=1)

events = parse_events(all_file_contents)



Parsing 956 raw events


In [4]:
import sqlite3



def initialize_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            start TEXT,
            end TEXT,
            eventType TEXT
        )
    ''')
    conn.commit()
    return conn



db_path = 'events.db'
conn = initialize_database(db_path)


cursor = conn.cursor()
for event in events:
    cursor.execute('''
        INSERT INTO events (name, start, end, eventType)
        VALUES (?, ?, ?, ?)
    ''', (event['name'], event['start'].isoformat(), event['end'].isoformat(), event['eventType']))
conn.commit() 


